In [283]:
import pandas as pd
import numpy as np
import re

In [284]:
pd.options.display.max_columns = None

In [285]:
df1 = pd.read_csv('../dpe-69.csv')

/tmp/ipykernel_17187/3583860891.py:1: DtypeWarning: Columns (35,36,37,43,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('../dpe-69.csv')


In [286]:
df1.shape

(332834, 131)

In [287]:
df1.head()

,id,numero_dpe,usr_diagnostiqueur_id,usr_logiciel_id,tr001_modele_dpe_id,nom_methode_dpe,version_methode_dpe,nom_methode_etude_thermique,version_methode_etude_thermique,date_visite_diagnostiqueur,date_etablissement_dpe,date_arrete_tarifs_energies,commentaires_ameliorations_recommandations,explication_personnalisee,consommation_energie,classe_consommation_energie,estimation_ges,classe_estimation_ges,tr002_type_batiment_id,secteur_activite,tr012_categorie_erp_id,tr013_type_erp_id,annee_construction,surface_habitable,surface_thermique_lot,tv016_departement_id,commune,arrondissement,type_voie,nom_rue,numero_rue,batiment,escalier,etage,porte,code_postal,code_insee_commune,code_insee_commune_actualise,numero_lot,quote_part,nom_centre_commercial,surface_commerciale_contractuelle,portee_dpe_batiment,partie_batiment,shon,surface_utile,surface_thermique_parties_communes,en_souterrain,en_surface,nombre_niveaux,nombre_circulations_verticales,nombre_boutiques,presence_verriere,surface_verriere,type_vitrage_verriere,nombre_entrees_avec_sas,nombre_entrees_sans_sas,surface_baies_orientees_nord,surface_baies_orientees_est_ouest,surface_baies_orientees_sud,surface_planchers_hauts_deperditifs,surface_planchers_bas_deperditifs,surface_parois_verticales_opaques_deperditives,etat_avancement,organisme_certificateur,adresse_organisme_certificateur,dpe_vierge,est_efface,date_reception_dpe,longitude,latitude,geo_score,geo_type,geo_adresse,geo_id,geo_l4,geo_l5,tr001_modele_dpe_code,tr001_modele_dpe_type_id,tr001_modele_dpe_modele,tr001_modele_dpe_description,tr001_modele_dpe_fichier_vierge,tr001_modele_dpe_est_efface,tr001_modele_dpe_type,tr001_modele_dpe_type_libelle,tr001_modele_dpe_type_ordre,tr002_type_batiment_code,tr002_type_batiment_description,tr002_type_batiment_libelle,tr002_type_batiment_est_efface,tr002_type_batiment_ordre,tr002_type_batiment_simulateur,tr012_categorie_erp_code,tr012_categorie_erp_categorie,tr012_categorie_erp_groupe,tr012_categorie_erp_est_efface,tr013_type_erp_code,tr013_type_erp_type,tr013_type_erp_categorie_id,tr013_type_erp_est_efface,tr013_type_erp_categorie,tv016_departement_code,tv016_departement_departement,tv017_zone_hiver_id,tv018_zone_ete_id,tv016_departement_altmin,tv016_departement_altmax,tv016_departement_nref,tv016_departement_dhref,tv016_departement_pref,tv016_departement_c2,tv016_departement_c3,tv016_departement_c4,tv016_departement_t_ext_basse,tv016_departement_e,tv016_departement_fch,tv016_departement_fecs_ancienne_m_i,tv016_departement_fecs_recente_m_i,tv016_departement_fecs_solaire_m_i,tv016_departement_fecs_ancienne_i_c,tv016_departement_fecs_recente_i_c,tv017_zone_hiver_code,tv017_zone_hiver_t_ext_moyen,tv017_zone_hiver_peta_cw,tv017_zone_hiver_dh14,tv017_zone_hiver_prs1,tv018_zone_ete_code,tv018_zone_ete_sclim_inf_150,tv018_zone_ete_sclim_sup_150,tv018_zone_ete_rclim_autres_etages,tv018_zone_ete_rclim_dernier_etage
0,134,1369V1000001F,227,2,1,3CL - DPE,V2012,NaN,NaN,2013-04-10,2013-04-10,2011-08-15,Logement non occupé depuis plus d'une année. A...,NaN,416.56,F,21.73,D,1,Habitation (Maisons individuelles),NaN,NaN,1980,100.71,100.71,69,DÉCINES-CHARPIEU,50,rue,de Verdun,28,NaN,NaN,NaN,NaN,69150,69275,69275,sans objet,NaN,NaN,NaN,2.0,NaN,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Socotec certification,"3, av du Centre 78280 Guyancourt",0.0,0,2013-04-10 04:00:00,3.480600,46.828209,0.73,poi.bus_stop,Avenue de Verdun / Rue des Pendants (Arrêt de ...,https://osm.org/node/1804339871,NaN,NaN,V1,1,6.1,Pour les bâtiments à usage principal d’h...,1_V1_DPE_6.1.pdf,0,Vente,Vente,1,TR002_001,Maison Individuelle,Maison,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69,69 - Rhône,1,3,0,1200,4900,54000,80,340.0,1.5,NaN,-10,392.0,25.2,53.5,67.8,90.0,29.0,43.0,H1,6.58,10.5,42030,3.6,Ec,4,6,3.0,4
1,147,1369L1000001E,3825,3,9,3CL,"3CL-DPE, version 1.3",NaN,NaN,2013-04-09,2013-04-10,2011-09-15,NaN,NaN,254.00,E,47.00,E,1,Maison Individuelle,NaN,NaN,1978,97.00,97.00,69,COURS-LA-VILLE,NaN,NaN,13 

In [288]:
df1['annee'] = df1['date_etablissement_dpe'].apply(lambda x: str(x).split('-')[0])

In [289]:
print(df1.columns.tolist())

['id', 'numero_dpe', 'usr_diagnostiqueur_id', 'usr_logiciel_id', 'tr001_modele_dpe_id', 'nom_methode_dpe', 'version_methode_dpe', 'nom_methode_etude_thermique', 'version_methode_etude_thermique', 'date_visite_diagnostiqueur', 'date_etablissement_dpe', 'date_arrete_tarifs_energies', 'commentaires_ameliorations_recommandations', 'explication_personnalisee', 'consommation_energie', 'classe_consommation_energie', 'estimation_ges', 'classe_estimation_ges', 'tr002_type_batiment_id', 'secteur_activite', 'tr012_categorie_erp_id', 'tr013_type_erp_id', 'annee_construction', 'surface_habitable', 'surface_thermique_lot', 'tv016_departement_id', 'commune', 'arrondissement', 'type_voie', 'nom_rue', 'numero_rue', 'batiment', 'escalier', 'etage', 'porte', 'code_postal', 'code_insee_commune', 'code_insee_commune_actualise', 'numero_lot', 'quote_part', 'nom_centre_commercial', 'surface_commerciale_contractuelle', 'portee_dpe_batiment', 'partie_batiment', 'shon', 'surface_utile', 'surface_thermique_parti

In [290]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332834 entries, 0 to 332833
Columns: 132 entries, id to annee
dtypes: float64(49), int64(28), object(55)
memory usage: 335.2+ MB


In [291]:
df1.describe(include='all')

,id,numero_dpe,usr_diagnostiqueur_id,usr_logiciel_id,tr001_modele_dpe_id,nom_methode_dpe,version_methode_dpe,nom_methode_etude_thermique,version_methode_etude_thermique,date_visite_diagnostiqueur,date_etablissement_dpe,date_arrete_tarifs_energies,commentaires_ameliorations_recommandations,explication_personnalisee,consommation_energie,classe_consommation_energie,estimation_ges,classe_estimation_ges,tr002_type_batiment_id,secteur_activite,tr012_categorie_erp_id,tr013_type_erp_id,annee_construction,surface_habitable,surface_thermique_lot,tv016_departement_id,commune,arrondissement,type_voie,nom_rue,numero_rue,batiment,escalier,etage,porte,code_postal,code_insee_commune,code_insee_commune_actualise,numero_lot,quote_part,nom_centre_commercial,surface_commerciale_contractuelle,portee_dpe_batiment,partie_batiment,shon,surface_utile,surface_thermique_parties_communes,en_souterrain,en_surface,nombre_niveaux,nombre_circulations_verticales,nombre_boutiques,presence_verriere,surface_verriere,type_vitrage_verriere,nombre_entrees_avec_sas,nombre_entrees_sans_sas,surface_baies_orientees_nord,surface_baies_orientees_est_ouest,surface_baies_orientees_sud,surface_planchers_hauts_deperditifs,surface_planchers_bas_deperditifs,surface_parois_verticales_opaques_deperditives,etat_avancement,organisme_certificateur,adresse_organisme_certificateur,dpe_vierge,est_efface,date_reception_dpe,longitude,latitude,geo_score,geo_type,geo_adresse,geo_id,geo_l4,geo_l5,tr001_modele_dpe_code,tr001_modele_dpe_type_id,tr001_modele_dpe_modele,tr001_modele_dpe_description,tr001_modele_dpe_fichier_vierge,tr001_modele_dpe_est_efface,tr001_modele_dpe_type,tr001_modele_dpe_type_libelle,tr001_modele_dpe_type_ordre,tr002_type_batiment_code,tr002_type_batiment_description,tr002_type_batiment_libelle,tr002_type_batiment_est_efface,tr002_type_batiment_ordre,tr002_type_batiment_simulateur,tr012_categorie_erp_code,tr012_categorie_erp_categorie,tr012_categorie_erp_groupe,tr012_categorie_erp_est_efface,tr013_type_erp_code,tr013_type_erp_type,tr013_type_erp_categorie_id,tr013_type_erp_est_efface,tr013_type_erp_categorie,tv016_departement_code,tv016_departement_departement,tv017_zone_hiver_id,tv018_zone_ete_id,tv016_departement_altmin,tv016_departement_altmax,tv016_departement_nref,tv016_departement_dhref,tv016_departement_pref,tv016_departement_c2,tv016_departement_c3,tv016_departement_c4,tv016_departement_t_ext_basse,tv016_departement_e,tv016_departement_fch,tv016_departement_fecs_ancienne_m_i,tv016_departement_fecs_recente_m_i,tv016_departement_fecs_solaire_m_i,tv016_departement_fecs_ancienne_i_c,tv016_departement_fecs_recente_i_c,tv017_zone_hiver_code,tv017_zone_hiver_t_ext_moyen,tv017_zone_hiver_peta_cw,tv017_zone_hiver_dh14,tv017_zone_hiver_prs1,tv018_zone_ete_code,tv018_zone_ete_sclim_inf_150,tv018_zone_ete_sclim_sup_150,tv018_zone_ete_rclim_autres_etages,tv018_zone_ete_rclim_dernier_etage,annee
count,3.328340e+05,332834,332834.000000,332834.000000,332834.000000,332834,251448,55934,52632,332834,332834,332834,53090,29501,3.328340e+05,332834,3.328340e+05,332834,332834.000000,26224,2047.000000,16443.000000,332834.000000,332828.000000,332828.000000,332834.0,332614,117626,72243,332754,92482,26055,100146,47120,8884,332768,332768.0,332768.0,159813,11895,0.0,45027.000000,267208.000000,45,250526.000000,267208.000000,267208.000000,250526.0,250526.000000,267208.000000,250526.0,250526.0,267208.000000,250526.0,0.0,250526.0,250531.0,257683.000000,257683.000000,257683.000000,257683.000000,257683.000000,257683.000000,332834.0,331746,319712,307844.000000,332834.0,332834,310283.000000,310283.000000,332834.000000,310283,310283,306909,300444,829,332834,332834.000000,332834,332834,332834,332834.0,332834,332834,332834.000000,332834,332834,332834,332834.0,332834.000000,332834.000000,2047,2047,2047,2047.0,16443,16443,16443.000000,16443.0,16443,332834.0,332834,332834.0,332834.0,332834.0,332834.0,332834.0,332834.0,332834.0,332834.0,332834.0,0.0,332834.0,332834.0,3.328340e+05,332834.0,3.328340e+05,332

In [292]:
list_arrondissements = range(69381,69390,1)

In [293]:
# keeping only Lyon city and its arrondissements
dfi1 = pd.DataFrame()
mask = df1['code_insee_commune']==69123
df_temp = df1[mask]
dfi1 = pd.concat([dfi1, df_temp])
dfi1.shape
for i in list_arrondissements :
    mask = df1['code_insee_commune']==i
    df_temp = df1[mask]
    dfi1 = pd.concat([dfi1, df_temp])
print(dfi1.shape)

(94331, 132)


In [294]:
col_to_keep = ['annee', 'classe_consommation_energie', 'secteur_activite', 'annee_construction', 'surface_habitable', 'arrondissement', 'type_voie', 'nom_rue', 'code_insee_commune', 'numero_rue', 'batiment', 'escalier', 'etage', 'porte', 'nombre_niveaux', 'surface_baies_orientees_nord', 'surface_baies_orientees_est_ouest', 'surface_baies_orientees_sud', 'longitude', 'latitude', 'tr001_modele_dpe_type_libelle']

In [295]:
dfi1 = dfi1[col_to_keep]
dfi1.shape

(94331, 21)

In [296]:
dfi1['classe_consommation_energie'].unique()

array(['D', 'E', 'F', 'A', 'G', 'C', 'N', 'B'], dtype=object)

In [297]:
mask = dfi1['classe_consommation_energie']=='N'
dfi1 = dfi1[~mask]
dfi1.shape

(77626, 21)

In [298]:
dfi1['classe_consommation_energie'].unique()

array(['D', 'E', 'F', 'A', 'G', 'C', 'B'], dtype=object)

In [299]:
dfi1['secteur_activite'].unique()

array(['Habitation (parties privatives d immeuble col', nan,
       "Habitation (Parties privatives d'immeuble col",
       'Habitation (maisons individuelles)', 'Bureaux',
       'Habitation (Maisons individuelles)', 'Local commercial', 'Garage',
       'Residence', 'Habitation (Parties communes)', 'Commerce',
       'Habitation (parties communes)', 'Autres',
       "Immeuble d'habitation collectif",
       'Batiment(s) a usage de commerces et d habitat',
       'Batiment a usage de bureaux et d habitation',
       'Etablissements sociaux',
       'Habitation (maison en copropriete horizontale',
       'Chambres privatives', 'Appartement', 'Etablissement sociaux',
       'Habitation (maison individuelle)', 'Maison individuelle',
       'Centre commercial'], dtype=object)

In [300]:
activite_to_dump = ['Bureaux',
       'Local commercial', 'Garage',
       'Commerce',
       'Autres',
       'Etablissements sociaux',
       'Chambres privatives', 'Etablissements sociaux', 'Etablissement sociaux', 'Centre commercial']

In [301]:
dfi1['mask_activite'] = dfi1['secteur_activite'].map(lambda x : 0 if x in activite_to_dump else 1)
mask = dfi1['mask_activite']==1
dfi1 = dfi1[mask]
dfi1.shape

(77602, 22)

In [302]:
dfi1['tr001_modele_dpe_type_libelle'].unique()

array(['Location', 'Copropriété', 'Vente', 'Neuf', 'Bâtiment public'],
      dtype=object)

In [303]:
mask = dfi1['tr001_modele_dpe_type_libelle'] == 'Vente'
dfi1 = dfi1[mask]
dfi1.shape

(31168, 22)

In [304]:
dfi1['code_insee_commune'] = dfi1['code_insee_commune'].astype(int)

In [305]:
dfi1['code_insee_commune'].unique()

array([69123, 69381, 69382, 69383, 69384, 69385, 69386, 69387, 69388,
       69389])

In [306]:
#determine orientation
def orientation(n, eo, s):
    try :
        maxi = max(n, eo, s)
    except :
        return None

    if maxi == 0 :
        return None
    if maxi == n :
        return 'N'
    if maxi == eo :
        return 'EO'
    if maxi == s :
        return 'S'
    


In [307]:
dfi1['orientation'] = list(map(orientation, dfi1['surface_baies_orientees_nord'], dfi1['surface_baies_orientees_est_ouest'], dfi1['surface_baies_orientees_sud']))

In [308]:
dfi1['orientation'].value_counts()

EO    7500
S     3835
N     3410
Name: orientation, dtype: int64

In [309]:
dfi1['orientation'].isna().sum()

16423

In [310]:
dfi1['annee'] = pd.to_numeric(dfi1['annee'])
dfi1['annee'] = dfi1['annee'].astype(int)

In [311]:
#determine code postal
list_keys = [69123, 69381, 69382, 69383, 69384, 69385, 69386, 69387, 69388,69389]
list_values = [69000, 69001, 69002, 69003, 69004, 69005, 69006, 69007, 69008,69009]
dict_code = dict(zip(list_keys, list_values))
def code_postal(c) :
    return dict_code[c]

In [312]:
dfi1['code_postal'] = dfi1['code_insee_commune'].apply(code_postal)

In [313]:
#create column 'rue' to gather type_voie and nom_rue
def rue(type, nom) :
    if pd.notnull(type) and pd.notnull(nom) :
        return type + ' ' + nom
    if pd.notnull(type)==False :
        return nom
    if pd.notnull(nom)==False :
        return type

In [314]:
dfi1['type_voie'] = dfi1['type_voie'].astype(str)
dfi1['nom_rue'] = dfi1['nom_rue'].astype(str)
dfi1['rue'] = list(map(rue, dfi1['type_voie'], dfi1['nom_rue'])) # permet de concaténer sans récupérer les nan

In [315]:
dfi1['etage'].unique()

array(['1er', '3eme Etage', '1er Etage', 'NC', '2eme Etage', nan,
       '1er Etage sur Entresol', 'RDC', '4eme Etage', '5eme Etage',
       '6eme Etage', '1', '5ème', '6', 'RDC ', '03', '2', '04', 'rdc ',
       '3ème étage', '02', 'Rez de jardin', '9eme Etage',
       'Rez de chaussee', '3ème', '1ER', '10eme Etage', '2ème', '4ème',
       '1er SS', '8', '07', '4', '06', '5ème étage', '05', '01', '15',
       '1er étage', '2ème ', '2ème étage', 'Rez de chaussée', '4ème ',
       '4ème étage', '3', '12ème', '7eme Etage', '8eme Etage', '4eme',
       '2ème étage sur entresol', '6ème', 'RDJ', '2eme', '10', '7ème',
       '3eme', '8ème', '2ème étage SUR ENTRESOL', 'ENTRE', '08', '5',
       'Rez de chaussee sureleve', '7', '+ 3.5', '12eme Etage', '7eme',
       '6ème étage', 'Entresol', 'RRDJ', '5eme', '6eme',
       '4eme Etage sur Entresol', '14eme Etage', 'rdc', 'Non communiqué',
       '5eme et 6eme Etage', 'RDC G', 'RDC D', '1ER G', '2G', '-1', 'R+1',
       '12', '3eme et 4eme Etage

In [316]:
#récupération de l'étage mal renseigné dans le fichier en vérifiant certains mots-clés et en isolant les numéros
dfi1['Etage_num'] = dfi1['etage'].apply(lambda x: list(map(int, re.findall(r'\d+', str(x))))) # isolate numbers
dfi1['RDC'] = dfi1['etage'].apply(lambda x: re.findall(r"rdc|rez de chaussee", str(x).lower())) # find rez de chaussee

In [317]:
def etage1(num, RDC) :
    if len(RDC) > 0 :    # si on a identifié RDC, alors l'étage est forcément au rez-de-chaussée
        return 0
    if len(num) > 0 : # si on a au moins 1 numéro, on prend le 1er
        return num[0]
    else :
        return float('NaN')

In [318]:
dfi1['etage'] = list(map(etage1, dfi1['Etage_num'], dfi1['RDC']))

In [319]:
dfi1.describe(include='all')

,annee,classe_consommation_energie,secteur_activite,annee_construction,surface_habitable,arrondissement,type_voie,nom_rue,code_insee_commune,numero_rue,batiment,escalier,etage,porte,nombre_niveaux,surface_baies_orientees_nord,surface_baies_orientees_est_ouest,surface_baies_orientees_sud,longitude,latitude,tr001_modele_dpe_type_libelle,mask_activite,orientation,code_postal,rue,Etage_num,RDC
count,31168.000000,31168,3042,31168.000000,31168.000000,4841,31168,31168,31168.000000,6721,760,16536,3422.000000,844,30350.000000,28828.000000,28828.000000,28828.000000,30907.000000,30907.000000,31168,31168.0,14745,31168.000000,31168,31168,31168
unique,NaN,7,12,NaN,NaN,25,116,20323,NaN,709,154,3345,NaN,245,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,3,NaN,21037,21,3
top,NaN,D,Habitation (parties privatives d immeuble col,NaN,NaN,00,nan,Jules VERNE,NaN,20,NC,Etage 1,NaN,Gauche,NaN,NaN,NaN,NaN,NaN,NaN,Vente,NaN,EO,NaN,rue Jules VERNE,[],[]
freq,NaN,12011,2239,NaN,NaN,822,24804,154,NaN,244,311,1224,NaN,176,NaN,NaN,NaN,NaN,NaN,NaN,31168,NaN,7500,NaN,153,28113,30796
mean,2017.345547,NaN,NaN,1848.572992,113.215006,NaN,NaN,NaN,69174.087076,NaN,NaN,NaN,2.943892,NaN,1.150214,1.680280,3.060188,1.571181,4.842422,45.757101,NaN,1.0,NaN,69001.027817,NaN,NaN,NaN
std,2.156311,NaN,NaN,471.613210,465.656422,NaN,NaN,NaN,103.877582,NaN,NaN,NaN,2.150792,NaN,1.666522,33.785754,10.173962,5.526032,0.052073,0.040443,NaN,0.0,NaN,2.347644,NaN,NaN,NaN
min,2000.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,69123.000000,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.368634,43.663685,NaN,1.0,NaN,69000.000000,NaN,NaN,NaN
25%,2016.000000,NaN,NaN,1947.000000,46.820000,NaN,NaN,NaN,69123.000000,NaN,NaN,NaN,1.000000,NaN,1.000000,0.000000,0.000000,0.000000,4.827331,45.746387,NaN,1.0,NaN,69000.000000,NaN,NaN,NaN
50%,2018.000000,NaN,NaN,1960.000000,66.450000,NaN,NaN,NaN,69123.000000,NaN,NaN,NaN,3.000000,NaN,1.000000,0.000000,0.000000,0.000000,4.844901,45.755725,NaN,1.0,NaN,69000.000000,NaN,NaN,NaN
75%,2019.000000,NaN,NaN,1989.000000,87.000000,NaN,NaN,NaN,69123.000000,NaN,NaN,NaN,4.000000,NaN,1.000000,0.000000,4.160000,0.000000,4.863324,45.768485,NaN,1.0,NaN,69000.000000,NaN,NaN,NaN


In [320]:
dfi1.groupby(['annee']).size()

annee
2000       2
2012       3
2013     691
2014    3859
2015    2789
2016    3659
2017    4308
2018    4485
2019    4989
2020    5598
2021     785
dtype: int64

In [321]:
list_to_keep = ['annee', 'classe_consommation_energie','secteur_activite', 'annee_construction', 'surface_habitable', 'numero_rue', 'rue', 'code_postal', 'etage', 'nombre_niveaux', 'orientation']
dfi1 = dfi1[list_to_keep]

In [322]:
list_columns_dpe = ['annee_dpe', 'diag_dpe','type_bat_dpe', 'annee_construction_dpe', 'surface_habitable_dpe', 'num_rue_dpe', 'rue_dpe', 'code_postal_dpe', 'etage_dpe', 'nbe_niveaux_dpe', 'orientation_dpe']

In [323]:
dfi1.columns = list_columns_dpe

In [324]:
dfi1.head()

,annee_dpe,diag_dpe,type_bat_dpe,annee_construction_dpe,surface_habitable_dpe,num_rue_dpe,rue_dpe,code_postal_dpe,etage_dpe,nbe_niveaux_dpe,orientation_dpe
4143,2013,G,Habitation (Parties privatives d'immeuble col,1995,19.28,8,rue de l'Epargne,69000,1.0,0.0,None
4206,2013,E,Habitation (parties privatives d immeuble col,1974,68.00,29,RUE RENE LEYNAUD,69000,3.0,1.0,None
4254,2013,E,Habitation (parties privatives d immeuble col,2000,32.00,27,RUE DES GRANGES,69000,1.0,1.0,S
4262,2013,D,NaN,2000,38.23,NC,nan 60 Boulevard Pinel Bâtiment B,69000,NaN,1.0,S
4268,2013,E,Habitation (parties privatives d immeuble col,1974,66.11,137,RUE de Montagny,69000,2.0,1.0,None
